In [35]:
'''Visualization of the filters of simple_CNN, via gradient ascent in input space.

Results example: http://i.imgur.com/4nj4KjN.jpg
'''
from __future__ import print_function
import os, sys
import inspect

#from scipy.misc import imsave
import numpy as np
import time
from keras import backend as K
from keras.models import load_model
os.environ["THEANO_FLAGS"] = 'exception_verbosity=high'

In [2]:
train_hist = '/data/user/tglauch/ML_Reco/train_hist/'
today = '2017-07-25'
project = 'highE'
project_path = os.path.join(train_hist, today, project)

In [31]:
# dimensions of the generated pictures for each filter.
img_width = 21
img_height = 21

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'conv3d_1'

# util function to convert a tensor into a valid image


def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [4]:
# build the network with energy reco weights
model = load_model(os.path.join(project_path, 'final_network.h5'))
print('Model loaded.')

model.summary()

Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 1, 7, 7, 32)       204032    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 7, 7, 32)       128       
_________________________________________________________________
activation_1 (Activation)    (None, 1, 7, 7, 32)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 7, 7, 32)       0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 1, 7, 7, 64)       256064    
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 7, 7, 64)       256       
_________________________________________________________________
activation_2 (Activation)    (None, 1, 7, 7, 64)       0      

In [5]:
# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[0:]])
print(layer_dict.keys())

[u'activation_1', u'dropout_1', u'activation_3', u'activation_2', u'dense_2', u'flatten_1', u'dropout_3', u'conv3d_2', u'conv3d_3', u'dropout_2', u'conv3d_1', u'dense_1', u'dropout_4', u'batch_normalization_2', u'batch_normalization_3', u'batch_normalization_1']


In [33]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

In [54]:
kept_filters = []
for filter_index in range(0, 32):
    # we only scan through the first 200 filters,
    # but there are actually 32 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output

    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 1, img_width, img_height, 51))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 51, 1))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

Processing filter 0


ValueError: Corr3dMM images and kernel must have the same stack size

Apply node that caused the error: Corr3dMM{half, (3, 3, 3), (1, 1, 1)}(InplaceDimShuffle{0,4,1,2,3}.0, Subtensor{::, ::, ::int64, ::int64, ::int64}.0)
Toposort index: 16
Inputs types: [TensorType(float32, 5D), TensorType(float32, 5D)]
Inputs shapes: [(1, 1, 21, 21, 51), (32, 51, 5, 5, 5)]
Inputs strides: [(89964, 89964, 4284, 204, 4), (4, 128, -163200, -32640, -6528)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[InplaceDimShuffle{0,2,3,4,1}(Corr3dMM{half, (3, 3, 3), (1, 1, 1)}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/home/jkager/.local/lib/python2.7/site-packages/keras/models.py", line 314, in model_from_config
    return layer_module.deserialize(config, custom_objects=custom_objects)
  File "/home/jkager/.local/lib/python2.7/site-packages/keras/layers/__init__.py", line 54, in deserialize
    printable_module_name='layer')
  File "/home/j

In [57]:
input_img

/conv3d_1_input

In [39]:
# we will stich the best 25 filters on a 5 x 5 grid.
n = 5

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 1
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img

# save the result to disk
imsave('stitched_filters_%dx%d.png' % (n, n), stitched_filters)

IndexError: list index out of range